In [2]:
import os
import json
import re
import sys
import unicodedata
import nltk
from nltk.corpus import stopwords
from classifier import *

In [3]:
class FileStorage():

    def __init__(self,filename):
        self.filename = filename

    def read(self):
        if os.path.exists(self.filename):
            with open(self.filename) as file:
                data = json.load(file)
                return data
        else:
            return {}
        
    def save(self,data):#write data like json file
        try:
            old_data = self.read()
            if len(old_data.keys()) == 0:
                old_data["tweets"] = []
            old_data["tweets"].append(data)
            jsondata = json.dumps(old_data, indent=4, skipkeys=True, sort_keys=True)
            fd = open(self.filename, 'w')
            fd.write(jsondata)
            fd.close()
            print (self.filename + " ha sido escrito exitosamente")
        except Exception as e:
            print (e)
            print ('ERROR writing', self.filename)


In [4]:
def process(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"(https?)\S+","",tweet)#remove urls
    tweet = re.sub(r"(\B#)\w*","",tweet)#remove hashtags
    tweet = re.sub(r"(\B@)\w*","",tweet)#remove mentions
    tweet = re.sub("\n","",tweet)#remove lines separate
    tweet = unicodedata.normalize('NFKD', tweet).encode('ASCII', 'ignore')#remove accents
    tweet = re.sub('[^a-zA-Z ]+', ' ', tweet.decode('ASCII'))#remove punctuactions
    text_no_stop_words = filter_stop_words(tweet)
    tokens = [token for token in text_no_stop_words if len(token) > 2 ]
    tweet = " ".join(tokens)
    return tweet




def filter_stop_words(text):
    stop_words_list = stopwords.words('spanish')
    stop_words_list += stopwords.words('english')
    text_filtered = [word for word in text.split() if word.lower() not in stop_words_list]
    return text_filtered

In [5]:
file_worker = FileStorage("buenos_aires.json")
data = file_worker.read()
print(data['tweets'][10])

{'contributors': None, 'coordinates': None, 'created_at': 'Sat Nov 28 14:52:09 +0000 2020', 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []}, 'favorite_count': 0, 'favorited': False, 'filter_level': 'low', 'geo': None, 'id': 1332698803904913410, 'id_str': '1332698803904913410', 'in_reply_to_screen_name': None, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'is_quote_status': False, 'lang': 'es', 'place': {'attributes': {}, 'bounding_box': {'coordinates': [[[-58.531792, -34.674453], [-58.531792, -34.534177], [-58.353494, -34.534177], [-58.353494, -34.674453]]], 'type': 'Polygon'}, 'country': 'Argentina', 'country_code': 'AR', 'full_name': 'Ciudad Autónoma de Buenos Aires, Argentina', 'id': '018f1cde6bad9747', 'name': 'Ciudad Autónoma de Buenos Aires', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/018f1cde6bad9747.json'}, 'quote_count': 0, 'reply_count': 0, 'ret

In [6]:
c = 0
clf = SentimentClassifier()

latitudes = []
longitudes = []
ids = []
created_at = []
users = []
rts = []
favs = []
texts = []
sent_scores = []
followers = []
links = []
for tweet in data['tweets']:
    if type(tweet['coordinates']) == dict:
        lat = tweet['coordinates']['coordinates'][1]
        lon = tweet['coordinates']['coordinates'][0]
        fecha_hora = tweet['created_at']
        id_ = tweet['id_str']
        text = tweet['text']
        text = process(text)
        favorite_count = tweet['favorite_count']
        retweet_count = tweet['retweet_count']
        followers_count = tweet['user']['followers_count']
        screen_name = tweet['user']['screen_name']
        link = 'https://twitter.com/' + screen_name + '/status/' + id_
        score = clf.predict(text)
        
        latitudes.append(lat)
        longitudes.append(lon)
        ids.append(id_)
        created_at.append(fecha_hora)
        texts.append(text)
        favs.append(favorite_count)
        rts.append(retweet_count)
        followers.append(followers_count)
        users.append(screen_name)
        links.append(link)
        sent_scores.append(score)
        c+=1

In [9]:
f = open('tweets_coordinates.csv', 'w')
f.write('id,created_at,latitude,longitude,user_screen_name,retweets_count,favorites_count,text,followers_count,link,sentiment_score\n')
for i in range(c):
    f.write(str(ids[i]) + ',' + created_at[i] + ',' + str(latitudes[i]) + ',' + str(longitudes[i]) + ',' + users[i] + ',' + str(rts[i]) + ',' + str(favs[i]) + ',' + texts[i] + ',' + str(followers[i]) + ',' + links[i] + ',' + str(sent_scores[i]) +'\n')                   
f.close()


In [ ]:
users[i] + ',' + rts[i] + ',' + favs[i] + ',' + texts[i] + ',' + followers[i] + ',' + links[i] + ',' + str(sent_scores[i]) +'\n')                   